In [ ]:
input_path = "/home/dani/dev/conny-dev/rental-multiclass/data/input/data.jsonl"

In [ ]:
import datasets

In [ ]:
cdataset = datasets.load_dataset("json", data_dir="/home/dani/dev/conny-dev/rental-multiclass/data/output/")

In [ ]:
N = len(cdataset["validation"])

In [ ]:
import random

In [ ]:
import re

PATTERNS = [
    "\w+berg",
    "allee",
    "str.",
    "strasse",
    "straße",
    "(?<!\d\d\d.)\d\d\d\d\d", # non telephone zip code
    "paul-lincke-ufer",
    "8c",
    "LINCKE-UFER",
    "neukölln",
    "damm",
    "germany",
    "berlin",
    "münchen",
    "munchen",
    "stuttgart",
    "deutschland",
    "potsdam",
    "bremen",
    "wiesbaden",
    "hanover",
    "schwerin",
    "düsseldorf",
    "dusseldorf",
    "mainz",
    "saarbrücken",
    "saarbrucken",
    "dresden",
    "magdeburg",
    "kiel",
    "erfunt",
    "postfach", # PO box,
    "adresse"
]

PATTERNS = re.compile(r"|".join(PATTERNS), flags=re.IGNORECASE)

PASS_PATTERNS = [
    "postbank",
    "amtsgericht",
    "\d\d\.\d\d\.\d\d\d\d"
]

PASS_PATTERNS = [re.compile(pat, flags=re.IGNORECASE) for pat in PASS_PATTERNS]


In [ ]:
import conny_ml
from conny_ml.extraction import entities

In [ ]:
# parser = conny_ml.extraction.entities.RegexParser({"address_element": PATTERNS}, group=0)

def has_address(text):
    return bool(PATTERNS.search(text))

In [ ]:
from address_ner.address_generator import AddressGenerator

In [ ]:
generator = AddressGenerator()

In [ ]:
generator.sample()

In [ ]:
datasets.load_dataset?

In [ ]:
X = datasets.load_dataset("../data/output/")

In [ ]:
X["train"].features["ner_tags"].feature

In [ ]:
from collections import Counter

In [ ]:
len([a["properties"]["unit"] for a in generator.addresses if a["properties"]["unit"]])

In [ ]:
fake.building_number()

In [ ]:
fake.postcode()

In [ ]:
fake.city_name()

In [ ]:
fake.city()

In [ ]:
fake.building_number()

In [ ]:
len(generator.addresses)

In [ ]:
2 ** -1

In [ ]:
import random

In [ ]:
(10000000*(2 ** (-np.arange(26, dtype=float)))).astype(int).tolist()

In [ ]:
%%time
random.sample(range(26), counts = (1000000*(2 ** (-np.arange(26, dtype=float)))).astype(int).tolist(), k=1)

In [ ]:
Counter([a["properties"]["unit"] for a in generator.addresses if a["properties"]["unit"]])

In [ ]:
def generate_address():
    return generator.sample()

In [ ]:
char_kde = pd.read_pickle("char_kde.pkl")

In [ ]:
%%time
character_distribution = char_kde.resample(1000).flatten()
character_distribution[character_distribution<0] = 0
character_distribution = character_distribution.astype(int)

In [ ]:
import time

In [ ]:
def sample(text, character_distribution=[10], generator=None):
    """
    process:
        - split text by \n
        - remove lines that are "address-like"
        - collect lines into lists until length is just longer than a sampled character limit
        - generate an address for each list 
        - insert address uniformly at random each list
        - randomly join each list of lines with \n and \s
    """
    
    lines = text.split("\n")
    lines = [line for line in lines if not has_address(line)]

    if len(lines) == 0:
        return []
    
    # split into reasonable sum-samples
    samples = []
    sample = []
    length = random.choice(character_distribution)
    while lines:
        sample.append(lines.pop(0))
        
        if np.sum([len(line) for line in sample]) >= length:
            samples.append(sample)
            sample = []
            length = random.choice(character_distribution)
    
    # collect last sample regardless of length
    if sample:
        samples.append(sample)

    # generate addresses and insert uniformly at random
    entities = []
    for sample in samples:
        address = generator()
        index = random.randint(0, len(sample))
        sample.insert(index, address)
        
        entity = {}
        entity["text"] = address
        entity["begin"] = int(np.sum([len(line) for line in sample[:index]]) + len(sample[:index])) # lens of strings + joins
        entity["end"] = int(entity["begin"] + len(entity["text"]))
        entity["label"] = "address"
        
        entities.append(entity)
    
    # randomly join sample text with " " or "\n"
    for i, sample in enumerate(samples):

        sample_text = ""
        while len(sample) > 1:
            sample_text += sample.pop(0)
            sample_text += random.choice([" ", "\n"])
        sample_text += sample.pop() # add last string
        
        samples[i] = sample_text
        
    
    # sanity check
    for ent, sample in zip(entities, samples):
        assert sample[ent["begin"]:ent["end"]] == ent["text"]
    
    return list(zip(samples, entities))

In [ ]:
%%time

idx = random.randint(0, N)
text = cdataset["validation"][idx]["text"]
samples = sample(text, generator=generate_address, character_distribution=character_distribution)
# spans

In [ ]:
plt.hist(character_distribution, bins=50);

In [ ]:
plt.hist([len(generator.sample()) for i in range(10000)], bins=50);

In [ ]:
np.array('(50,5)', dtype=int)

In [ ]:
sub = cdataset.shuffle()
sub = sub.filter(lambda data, idx: idx < 1000, with_indices=True)

In [ ]:
from joblib import Parallel, delayed

In [ ]:
from itertools import chain

In [ ]:
train_ents = Parallel(n_jobs=8)(delayed(sample)(
    datum["text"],
    character_distribution,
    generate_address) for datum in sub["train"])


In [ ]:
train_ents = list(chain(*train_ents))
len(train_ents)

In [ ]:
# sentence size (capped at 500 chars)
plt.hist([len(s[0]) for s in train_ents], bins=np.arange(0,500));

In [ ]:
# ratio of ent text to sentence text
plt.hist([len(datum[1]["text"]) / len(datum[0]) for datum in train_ents], bins=50);

In [ ]:
# len of ent text
plt.hist([len(datum[1]["text"]) for datum in train_ents], bins=50);

In [ ]:
# CONVERT TO CONLL FORMAT
# NEED TO MAP CHARACTER OFFSETS TO TOKENS

# USE SPACY

In [ ]:
from spacy.training import offsets_to_biluo_tags

In [ ]:
from spacy.training import offsets_to_biluo_tags, biluo_tags_to_spans
from conny_ml.extraction.entities.utils import resolve_overlap, align_entities

In [ ]:
import spacy
def get_spacy_de_model(model="de_core_news_sm"):

    exclude = [
        'tok2vec',
        'tagger',
        'morphologizer',
        'parser',
        'attribute_ruler',
        'lemmatizer',
        'ner'
    ]

    try:
        nlp = spacy.load(model, exclude=exclude)
    except OSError as error:
        import subprocess

        command = f'poetry run python -m spacy download {model}'
        process = subprocess.Popen(command.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()

    nlp = spacy.load(model, exclude=exclude)
    return nlp


In [ ]:
nlp = get_spacy_de_model()

In [ ]:
def sample_to_conll(sample, nlp=None, scheme="IOB"):
    assert scheme in ["IOB", "BILUO"]
    
    nlp = nlp or get_spacy_de_model(model="de_core_news_sm")
    doc = nlp(sample[0])
    doc_ents = align_entities([sample[1]], doc)
    doc_ents = resolve_overlap(doc_ents)
    
    tags = offsets_to_biluo_tags(doc, [(e["begin"], e["end"], e["label"]) for e in doc_ents])
    if scheme == "IOB":
        for i, tag in enumerate(tags):
            if tag[0] == "U":
                tags[i] = "B" + str(tag)[1:]
            if tag[0] == "L":
                tags[i] = "I" + str(tag)[1:]
                
    toks = [str(tok) for tok in doc]
    
    return {"tokens": toks, "ner_tags": tags}

In [ ]:
%%time
idx = random.randint(0, len(train_ents))
sample_to_conll(train_ents[idx], nlp=nlp, scheme="IOB")

In [ ]:
random.seed(42)

In [ ]:
random.randint(0, 10)

In [ ]:
def batch_to_conll(batch, random_state=42, scheme="IOB"):
    nlp = get_spacy_de_model(model="de_core_news_sm")
    
    results = []
    for datum in batch:
        results.append(sample_to_conll(datum, nlp=nlp, scheme=scheme))
    return results

In [ ]:
from multiprocessing import Pool
from functools import partial

In [ ]:
random_state = 42
n_workers = 8

In [ ]:
f = partial(batch_to_conll, random_state=random_state)

batches = np.array_split(train_ents, n_workers)

In [ ]:
%%time
OUT = Parallel(n_jobs=n_workers)(delayed(batch_to_conll)(batch) for batch in batches)

In [ ]:
OUT = list(chain(*OUT))
len(OUT)

In [ ]:
from collections import defaultdict

def flatten_dicts(dicts):
    out = defaultdict(list)
    keys = list(set(chain(*(tuple(d.keys()) for d in OUT))))
    for d in dicts:
        for k in keys:
            out[k].append(d.get(k))
    return dict(out)

In [ ]:
from datasets import Dataset

In [ ]:
train_data = Dataset.from_dict(flatten_dicts(OUT))
train_data

In [ ]:
labels = list(set(chain(*train_data.map(lambda sample: {"labels": list(set(sample["ner_tags"]))})["labels"])))
labels

In [ ]:
labels.remove("O")

In [ ]:
labels.sort(key=lambda label: label[::-1])
labels = ["O"] + labels

In [ ]:
labels

# HUGGING FACE

In [ ]:
import datasets

In [ ]:
dataset = datasets.load_dataset("conll2003")

In [ ]:
from pathlib import Path

In [ ]:
p = Path("../data/output/o")

In [ ]:
p.is_dir()

In [ ]:
p.parent

In [ ]:
dataset = _

In [ ]:
dataset["train"].to_json

In [ ]:
dataset["train"].to_json

In [ ]:
MODEL = "distilbert-base-multilingual-cased"
# MODEL = "Davlan/distilbert-base-multilingual-cased-ner-hrl"

In [ ]:
import transformers

In [ ]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-multilingual-cased"
batch_size = 16

In [ ]:
from datasets import load_dataset, load_metric, ClassLabel, Sequence

In [ ]:
train_data.features["ner_tags"] = Sequence(ClassLabel(num_classes=len(labels), names=labels))

In [ ]:
tag2id = {k:i for i, k in enumerate(labels)}

In [ ]:
train_data = train_data.map(lambda sample: {"ner_tags": [tag2id[tag] for tag in sample["ner_tags"]]})

In [ ]:
train_data[0]

In [ ]:
from faker import Faker

In [ ]:
fake = Faker("de")

In [ ]:
Faker.seed?

In [ ]:
x = "cat"

In [ ]:
x.upper()

In [ ]:
label_list = train_data.features[f"{task}_tags"].feature.names
label_list

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
tokenized_input = tokenizer(train_data[4]["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else tokenize_and_align_labels(datasets['train'][:5])[4][f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(train_data[:5])

In [ ]:
tokenized_datasets = train_data.map(tokenize_and_align_labels, batched=True)

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

In [ ]:
labels = [label_list[i] for i in train_data[4][f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from functools import partial

In [ ]:
train_small = tokenized_datasets.shard(100, index=0)
train_small

In [ ]:
val_small = tokenized_datasets.shard(100, index=1)
val_small

In [ ]:
test_small = tokenized_datasets.shard(100, index=2)
test_small

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_small,#["train"],
    eval_dataset=val_small,#["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
4

In [ ]:
X = tokenized_datasets.shard(1000, 999)

In [ ]:
predictions, labels, _ = trainer.predict(X)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
idx = 1

In [ ]:
idx += 1
X["tokens"][idx]

In [ ]:
list(zip(true_predictions[idx], true_labels[idx]))